In [ ]:
!mkdir -p .data .output

In [ ]:
import pandas as pd

#getting train datatset of twitter volume
url = "https://raw.githubusercontent.com/numenta/NAB/master/data/realTweets/Twitter_volume_AMZN.csv"
df = pd.read_csv(url, header=0, index_col=0)
df.to_csv('.data/dataset.csv')

In [ ]:
import sagemaker 

session = sagemaker.Session()
bucket = session.default_bucket()
key_prefix = 'frameworkprocessors/mxnet-example'

source_path = session.upload_data('.data', bucket=bucket, key_prefix=f'{key_prefix}/data')
source_path

In [ ]:
from sagemaker.mxnet import MXNetProcessor
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker import get_execution_role

mxp = MXNetProcessor(
    framework_version='1.8.0',
    py_version='py37',
    role=get_execution_role(), 
    instance_count=1,
    instance_type='ml.c5.xlarge',
    base_job_name='frameworkprocessor-mxnet'
)

In [ ]:
mxp.run(
    code='mxnet-gluonts-processing.py',
    source_dir='.',
    inputs=[
        ProcessingInput(
            input_name='data', source=source_path,
            destination='/opt/ml/processing/input/data/'
        )
    ],
    outputs=[
        ProcessingOutput(output_name='processed_data', source='/opt/ml/processing/output/')
    ],
    logs=False
)

In [ ]:
output_path = mxp.latest_job.outputs[0].destination

!aws s3 ls --recursive $output_path